In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/combined_dataset/goemotions_combined.csv')

In [3]:
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df.shape

(211225, 37)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211225 entries, 0 to 211224
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   text                  211225 non-null  object 
 1   id                    211225 non-null  object 
 2   author                211225 non-null  object 
 3   subreddit             211225 non-null  object 
 4   link_id               211225 non-null  object 
 5   parent_id             211225 non-null  object 
 6   created_utc           211225 non-null  float64
 7   rater_id              211225 non-null  int64  
 8   example_very_unclear  211225 non-null  bool   
 9   admiration            211225 non-null  int64  
 10  amusement             211225 non-null  int64  
 11  anger                 211225 non-null  int64  
 12  annoyance             211225 non-null  int64  
 13  approval              211225 non-null  int64  
 14  caring                211225 non-null  int64  
 15  

In [6]:
unwanted_cols = ['id', 'author','subreddit', 'link_id','parent_id','created_utc', 'rater_id']

In [7]:
df.drop(columns=unwanted_cols, inplace=True)

In [8]:
df.head()

,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,False,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
df= df[df['example_very_unclear']==False]
df.shape

(207814, 30)

In [10]:
df.dropna(inplace=True)

In [11]:
df.drop(columns=['example_very_unclear'], inplace=True)

In [12]:
X = df['text']
X.head()

0                                      That game hurt.
2       You do right, if you don't care then fuck 'em!
3                                   Man I love reddit.
4    [NAME] was nowhere near them, he was by the Fa...
5    Right? Considering it’s such an important docu...
Name: text, dtype: object

In [13]:
y= df.drop(columns=['text'])

In [14]:
y

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211220,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
211221,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211222,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y = df.iloc[:,1:]
y.head()

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y.dtypes

admiration        int64
amusement         int64
anger             int64
annoyance         int64
approval          int64
caring            int64
confusion         int64
curiosity         int64
desire            int64
disappointment    int64
disapproval       int64
disgust           int64
embarrassment     int64
excitement        int64
fear              int64
gratitude         int64
grief             int64
joy               int64
love              int64
nervousness       int64
optimism          int64
pride             int64
realization       int64
relief            int64
remorse           int64
sadness           int64
surprise          int64
neutral           int64
dtype: object

In [18]:
X.shape, y.shape

((207814,), (207814, 28))

In [19]:
print(X.sample(5))

86413     No. I make jokes about it, and so does he but ...
143556               Burn wait someone already did it sorry
66250     I hadn’t even noticed the hairline, was too bu...
170269    I’m now a [NAME] fan! Hope he does well, gl my...
73148                                         What an incel
Name: text, dtype: object


In [20]:
y.sample(5)

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
5809,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
129442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27967,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150795,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
def clean_text(text):
    